# ECE 143 Individual Project

Name: Yumeng Ruan  
PID: A53250267

# Problem

You have been asked to help with planning an ad-hoc communications network over a large rectangular region. Each individual tower can monitor a rectangular subsection of a specific width and height. The main problem is that none of the individual towers can provide coverage for the entire region of interest. Communications towers are unreliable and are put up independently and at random. You have no control over where or how big a tower’s footprint is placed. Importantly, due to technical issues such as cross-talk, no individual rectangular subsection can have multiple towers providing coverage for it. That is, there can be no overlap between any pair of rectangular subsections provided by the two respective towers. In any case, the desire is to maximize the coverage area of any available communications tower.

# Requirements

The order of when the towers come online is important. Once a tower has acquired its rectangular section, no subsequent tower can overlap that section. You may assume the following for this problem:

● All rectangular sections have integer-based corners.  
● All rectangular sections must be contained in the overall rectangular footprint.  
● The height and width of each rectangular section is sampled from a uniform distribution.  
● Positions of the windows are also determined by uniform random distribution.  
● All footprints must be rectangles (not general polygons).  
● When a new tower comes online, if its coverage rectangle intersects the pre-existing
composite footprint, then that new tower’s coverage is trimmed such that its maximum remaining coverage area is retained (see sequential diagram below).  

Write a detailed Jupyter notebook that implements a solution to this problem such that the user can supply the following overall size of desired coverage footprint and then determine the following:  

● Given an overall desired coverage footprint and a sequence of n communications towers, what   is the resulting resolved coverage?  
● What is the total area of coverage relative to the desired total coverage area of the         original footprint? That is, are there any gaps in coverage?  
● On average, how many communications towers are required before full coverage is obtained?  

# Discussion

Let's simplify this problem into 4 steps:  
1. At first, we can view a tower as a rectangle, we need a method to generate rectangles;  
2. Then input integer-based rectangles with random length and width sequently into a square region;    
3. Since we are required to check the overlapping section and then trim the wanting maximum non-overlapping rectangle, we need find the way;  
4. Repeat step3 until the whole square region is covered.

Follow these 4 steps we may find the result of the problem.

# Rectangle

In [43]:
class Rect(object):
    """The rectangle class
       A rectangle has four corners: x1,x2,x3,x4, where x1 represents bottom left,
       x2 represent bottom right, x3 represent top left and x4 represents top right."""
    
    def __init__(self, start, l, w):
        """Initialize function
           A legal rectangle can be determined by one start position plus length and width.
           We define the bottom left of a rectangle as the start position of it
           and length in x-axis and width in y-axis"""
        
        assert isinstance(start,tuple)
        assert isinstance(l,int)
        assert isinstance(w,int)
        assert l>0 and w>0
        
        self.x1 = start
        self.length = l
        self.width = w
    
    @property
    def x2(self):
        """Bottom right point"""
        return (self.x1[0] + self.length, self.x1[1])
    
    @property
    def x3(self):
        """Top left point"""
        return (self.x1[0], self.x1[1] + self.width)
    
    @property
    def x4(self):
        """Top right point"""
        return (self.x1[0] + self.length, self.x1[1] +self.width)
    
    def __repr__(self):
        return "Rectangle(%s,%s,%s,%s)" % (self.x1,self.x2,self.x3,self.x4)

    def __eq__(self,other):
        """We define equality of two rectangles if they are all the same"""
        return self.x1 == other.x1 and self.x2 == other.x2 and self.x3 == other.x3 and self.x4 == self.x4
    
    def __ge__(self,other):
        """We say one rectangle is greater or equal to another when it entirely contains another"""
        return self.x1[0]<=other.x1[0] and self.x1[1]<=other.x1[1] and self.x4[0]>=other.x4[0] and self.x4[1]>=other.x4[1]
    
    def __ne__(self,other):
        """We say two rectangles are inequal when they are not overlapped"""
        return self.x2[0] <= other.x1[0] or self.x1[0] >= other.x2[0] or self.x3[1] <= other.x1[1] or self.x1[1] >= other.x3[1]
    
    @property
    def area(self):
        """Calculate the area"""
        return self.length*self.width

In [40]:
r1 = Rect((3,3),3,3)
r2 = Rect((0,0),3,3)

In [42]:
r1.trans_x1((100,100))
r1

Rectangle((100, 100),(103, 100),(100, 103),(103, 103))

# Trim

In [44]:
import numpy as np
import itertools as it

def trim(R1,R2):
    """
       Trim two rectangles:
       if they are not overlapped, retain both;
       if the new one are contained entirely, set the new to 0;
       if they intersect, then that new rectangle’s coverage is trimmed 
       such that its maximum remaining coverage area is retained 
       
       """
    
    if R1 != R2:
        pass
    elif R1 >= R2:
        R2 = 0
    else:
        l1,r1,b1,t1 = R1.x1[0],R1.x2[0],R1.x1[1],R1.x3[1]
        l2,r2,b2,t2 = R2.x1[0],R2.x2[0],R2.x1[1],R2.x3[1]
        if not R1 != R2:
            if l2 > l1 and r2 > r1:
                if b2 < b1 and t2 > t1:
                    A,B,C = Rect((l2,t1),r2-l2,t2-t1),Rect((l2,b2),r2-l2,b1-b2),Rect((r1,b2),r2-r1,t2-b2)
                    max_area = max(A.area,B.area,C.area)
                    if A.area == max_area:
                        R2 = A
                    elif B.area == max_area:
                        R2 = B
                    else:
                        R2 = C
                elif b2 < b1 and t2 <= t1:
                    A,B = Rect((l2,b2),r2-l2,b1-b2),Rect((r1,b2),r2-r1,t2-b2)
                    if A.area > B.area:
                        R2 = A
                    else:
                        R2 = B
                elif b2 >=b1 and t2 > t1:
                    A,B= Rect((l2,t1),r2-l2,t2-t1),Rect((r1,b2),r2-r1,t2-b2)
                    if A.area > B.area:
                        R2 = A
                    else:
                        R2 = B
                else:
                    R2 = Rect((r1,b2),r2-r1,t2-b2)
            
            elif l2 < l1 and r2 < r1:
                if b2 < b1 and t2 > t1:
                    A,B,C = Rect((l2,t1),r2-l2,t2-t1),Rect((l2,b2),r2-l2,b1-b2),Rect((l2,b2),l1-l2,t2-b2)
                    max_area = max(A.area,B.area,C.area)
                    if A.area == max_area:
                        R2 = A
                    elif B.area == max_area:
                        R2 = B
                    else:
                        R2 = C
                elif b2 < b1 and t2 <= t1:
                    A,B = Rect((l2,b2),r2-l2,b1-b2),Rect((l2,b2),l1-l2,t2-b2)
                    if A.area > B.area:
                        R2 = A
                    else:
                        R2 = B
                elif b2 >=b1 and t2 > t1:
                    A,B= Rect((l2,t1),r2-l2,t2-t1),Rect((l2,b2),l1-l2,t2-b2)
                    if A.area > B.area:
                        R2 = A
                    else:
                        R2 = B
                else:
                    R2 = Rect((l2,b2),l1-l2,t2-b2)
            
            elif l2 < l1 and r2 > r1:
                if b2 < b1 and t2 > t1:
                    A,B,C,D = Rect((l2,t1),r2-l2,t2-t1),Rect((l2,b2),r2-l2,b1-b2),Rect((l2,b2),l1-l2,t2-b2),Rect((r1,b2),r2-r1,t2-b2)
                    max_area = max(A.area,B.area,C.area,D.area)
                    if A.area == max_area:
                        R2 = A
                    elif B.area == max_area:
                        R2 = B
                    elif C.area == max_area:
                        R2 = C
                    else:
                        R2 = D
                elif b2 < b1 and t2 <= t1:
                    A,B,C = Rect((l2,b2),r2-l2,b1-b2),Rect((l2,b2),l1-l2,t2-b2),Rect((r1,b2),r2-r1,t2-b2)
                    max_area = max(A.area,B.area,C.area)
                    if A.area == max_area:
                        R2 = A
                    elif B.area == max_area:
                        R2 = B
                    else:
                        R2 = C
                elif b2 >=b1 and t2 > t1:
                    A,B,C = Rect((l2,t1),r2-l2,t2-t1),Rect((l2,b2),l1-l2,t2-b2),Rect((r1,b2),r2-r1,t2-b2)
                    max_area = max(A.area,B.area,C.area)
                    if A.area == max_area:
                        R2 = A
                    elif B.area == max_area:
                        R2 = B
                    else:
                        R2 = C
                else:
                    A,B = Rect((l2,b2),l1-l2,t2-b2),Rect((r1,b2),r2-r1,t2-b2)
                    if A.area > B.area:
                        R2 = A
                    else:
                        R2 = B
            else:
                if b2 < b1 and t2 > t1:
                    A,B = Rect((l2,t1),r2-l2,t2-t1),Rect((l2,b2),r2-l2,b1-b2)
                    if A.area > B.area:
                        R2 = A
                    else:
                        R2 = B
                elif b2 < b1 and t2 <= t1:
                    R2 = Rect((l2,b2),r2-l2,b1-b2)
                else:
                    R2 = Rect((l2,t1),r2-l2,t2-t1)
    
    return R2        